In [3]:
import yfinance as yf
import bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [5]:
tesla = yf.Ticker('TSLA')

In [30]:
tesla_data = tesla.history()

In [31]:
tesla_data.reset_index(inplace = True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-11-10 00:00:00-05:00,189.899994,191.000000,180.029999,190.720001,132703000,0,0
1,2022-11-11 00:00:00-05:00,186.000000,196.520004,182.589996,195.970001,114403600,0,0
2,2022-11-14 00:00:00-05:00,192.770004,195.729996,186.339996,190.949997,92226600,0,0
3,2022-11-15 00:00:00-05:00,195.880005,200.820007,192.059998,194.419998,91293800,0,0
4,2022-11-16 00:00:00-05:00,191.509995,192.570007,185.660004,186.919998,66567600,0,0


In [32]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")

In [33]:
# tables = soup.find_all('table')
# for index,table in enumerate(tables):
#   if ("Tesla Quarterly Revenue" in str(table)):
#     table_index = index
# print(table_index)
tesla_revenue=pd.read_html(url, match="Tesla Quarterly Revenue", flavor='bs4')[0]
tesla_revenue.head()

,Tesla Quarterly Revenue(Millions of US $),Tesla Quarterly Revenue(Millions of US $).1
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


In [34]:
# tesla_revenue = pd.DataFrame(columns=['Date','Revenue'])
# for row in tables[table_index].tbody.find_all('tr'):
#   col = row.find_all('td')
#   if(col != []):
#     date = col[0].text
#     revenue = col[1].text
#     tesla_revenue = tesla_revenue.append({'Date':date, 'Revenue':revenue},ignore_index=True)
# tesla_revenue
tesla_revenue = tesla_revenue.rename(columns={"Tesla Quarterly Revenue(Millions of US $)":"Date","Tesla Quarterly Revenue(Millions of US $).1":"Revenue"}) #Rename df columns to 'Date' and 'Revenue'
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"") 
tesla_revenue.head() 

<ipython-input-34-c9b11ccead6a>:10: FutureWarning:

The default value of regex will change from True to False in a future version.



,Date,Revenue
0,2022-09-30,21454
1,2022-06-30,16934
2,2022-03-31,18756
3,2021-12-31,17719
4,2021-09-30,13757


In [35]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [36]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


Using yfinance to Extract Stock Data

In [37]:
game_stop = yf.Ticker("GME")

In [38]:
gme_data = game_stop.history(period="ytd")

In [39]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-01-03 00:00:00-05:00,37.307499,39.845001,37.302502,38.209999,5654800,0,0.0
1,2022-01-04 00:00:00-05:00,38.042500,38.247501,35.762501,37.227501,7339600,0,0.0
2,2022-01-05 00:00:00-05:00,37.097500,37.212502,32.287498,32.342499,13585600,0,0.0
3,2022-01-06 00:00:00-05:00,33.209999,34.424999,30.285000,32.757500,24206000,0,0.0
4,2022-01-07 00:00:00-05:00,39.942501,40.077499,33.125000,35.154999,48549200,0,0.0


Using Webscraping to Extract GME Revenue Data

In [40]:
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_data = requests.get(url).text

In [41]:
soup = BeautifulSoup(html_data,'html5lib')

In [42]:
gme_revenue=pd.read_html(url,match="GameStop Quarterly Revenue", flavor='bs4')[0]
#gme_revenue.head()
gme_revenue = gme_revenue.rename(columns={"GameStop Quarterly Revenue(Millions of US $)":"Date","GameStop Quarterly Revenue(Millions of US $).1":"Revenue"}) 
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"") 
gme_revenue.head() 

<ipython-input-42-d2155d9a98c1>:4: FutureWarning:

The default value of regex will change from True to False in a future version.



,Date,Revenue
0,2022-10-31,1186
1,2022-07-31,1136
2,2022-04-30,1378
3,2022-01-31,2254
4,2021-10-31,1297


In [43]:
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

gme_revenue.tail()

,Date,Revenue
51,2010-01-31,3524
52,2009-10-31,1835
53,2009-07-31,1739
54,2009-04-30,1981
55,2009-01-31,3492


In [44]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

In [45]:
make_graph(gme_data, gme_revenue, 'GameStop')